# 2.6 MLflow in practice

# Scenario 1: single data scientist participating in a ML competition  

**MLflow setup**:
* **Tracking server**: no
* **Backend store**:   local filesystem
* **Artifact store**:  local filesystem

---

In [1]:
import mlflow

No need to specify tracking URI: store in local filesystem

In [2]:
# default local filesystem
print(f'tracking URI: {mlflow.get_tracking_uri()}')

tracking URI: file:///workspaces/mlops-zoomcamp/02-experiment-tracking/04_use_case_scenarios/mlruns


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///workspaces/mlops-zoomcamp/02-experiment-tracking/04_use_case_scenarios/mlruns/0', creation_time=1737106246212, experiment_id='0', last_update_time=1737106246212, lifecycle_stage='active', name='Default', tags={}>]

search_experiments() creates *mlruns* directory with *'Default'* experiment in it

## Logistic regression in Iris Dataset  

Run on 'my-experiment-1'

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/01/17 09:30:46 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/01/17 09:30:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'file:///workspaces/mlops-zoomcamp/02-experiment-tracking/04_use_case_scenarios/mlruns/682501136300444253/3732085fe629499cb80e527e63078b71/artifacts'


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///workspaces/mlops-zoomcamp/02-experiment-tracking/04_use_case_scenarios/mlruns/682501136300444253', creation_time=1737106246583, experiment_id='682501136300444253', last_update_time=1737106246583, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///workspaces/mlops-zoomcamp/02-experiment-tracking/04_use_case_scenarios/mlruns/0', creation_time=1737106246212, experiment_id='0', last_update_time=1737106246212, lifecycle_stage='active', name='Default', tags={}>]

## Interact with model registry

In [6]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

In [10]:
from mlflow.exceptions import MlflowException

client.search_registered_models()

[]

It's not possible to use the model registry in the case we're using local filesystem as **backend store**.  
Can be solved by using different backend store like sqlite  